In [69]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely import wkt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [70]:
file_name = "RI_Congressional_2018.xlsx"
df = pd.read_excel(file_name)
df.head()

,City/Town,Precinct,Total Votes,Electon Day,Mail Ballots,Party,Contest,Candidate,Vote For,Unnamed: 9
0,Barrington 0101,101,2721,0,0,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,0,NaN
1,Barrington 0102,102,3229,0,0,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,0,NaN
2,Barrington 0103,103,2572,0,0,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,0,NaN
3,Barrington 0104,104,3147,0,0,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,0,NaN
4,Barrington 0105,105,3116,0,0,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,0,NaN


In [71]:
if "Contest" in list(df.columns):
    df.rename({"Contest":"Office Title"}, axis=1, inplace=True)
if "City/Town" in list(df.columns):
    df.rename({"City/Town":"Precinct Name"}, axis=1, inplace=True)
if "Electon Day" in list(df.columns):
    df.rename({"Electon Day":"E Day"}, axis=1, inplace=True)
if "Mail Ballots" in list(df.columns):
    df.rename({"Mail Ballots":"Mail"}, axis=1, inplace=True)
if "Precinct" in list(df.columns):
    df.rename({"Precinct":"Precinct #"}, axis=1, inplace=True)
if "Unnamed: 9" in list(df.columns):
    df.drop(columns=["Unnamed: 9"], axis=1, inplace=True)

cong = df[df["Office Title"].str.contains("Representative in Congress District")]
cong.head()

,Precinct Name,Precinct #,Total Votes,E Day,Mail,Party,Office Title,Candidate,Vote For
2315,Barrington 0101,101,916,858,58,DEM,Representative in Congress District 1,David N. Cicilline,1
2316,Barrington 0102,102,1245,1131,114,DEM,Representative in Congress District 1,David N. Cicilline,1
2317,Barrington 0103,103,991,873,118,DEM,Representative in Congress District 1,David N. Cicilline,1
2318,Barrington 0104,104,1176,1076,100,DEM,Representative in Congress District 1,David N. Cicilline,1
2319,Barrington 0105,105,1186,1067,119,DEM,Representative in Congress District 1,David N. Cicilline,1


In [72]:
real = gpd.read_file("../../../all_states_precincts_cname_neighbor_demo_OG_anomalous_fixNeighborVA.json")
real = real[real["State"] == "ri"]
real.head(1)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
2446,Bristol,0101,1027,522,60,23,2,1634,ri,ri-bristol-0101,...,3268,3055,23,3,110,73,None,None,None,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."


In [73]:
# fill missing party with NON
cong.fillna(value="NON", inplace=True)
cong.isna().sum()

Precinct Name    0
Precinct #       0
Total Votes      0
E Day            0
Mail             0
Party            0
Office Title     0
Candidate        0
Vote For         0
dtype: int64

# read another csv and map county to this dataset

In [74]:
df2 = pd.read_csv("../../../../sec data/voting/presidential_2016/processed data/RI_presidential_2016.csv")

In [75]:
df2.head()

,county,precinct,candidate,party,votes,county_lat,county_long
0,Bristol County,Barrington 0101; 101,D. Trump,republican,522,41.70683,-71.28664
1,Bristol County,Barrington 0101; 101,G. Johnson,libertarian,60,41.70683,-71.28664
2,Bristol County,Barrington 0101; 101,H. Clinton,democratic,1027,41.70683,-71.28664
3,Bristol County,Barrington 0101; 101,J. Stein,green,23,41.70683,-71.28664
4,Bristol County,Barrington 0101; 101,other,other,39,41.70683,-71.28664


In [76]:
c2p = {}
for index, row in df2.iterrows():
    county = row["county"]
    splited = row["precinct"].split(";")
    precinct1, precinct2 = splited[0].strip(), splited[1].strip()
    if precinct2 not in c2p.keys():
        c2p[precinct2] = [precinct1, county]
    else:
        if c2p[precinct2] != [precinct1, county]:
            print("Error", c2p[precinct2], [precinct1, county])
c2pCong = {}
for index, row in cong.iterrows():
    precinct1, precinct2 = row["Precinct Name"], row["Precinct #"]
    if precinct2 not in c2pCong.keys():
        c2pCong[precinct2] = precinct1
    else:
        if c2pCong[precinct2] != precinct1:
            print("Error", c2pCong[precinct2], precinct1)

In [77]:
cong = cong.astype({"Precinct #": "str"})

In [78]:
inDF2notCong = {}
for precinctNum, match in c2p.items():
    index = cong[(cong["Precinct #"] == precinctNum) & (cong["Precinct Name"] == match[0])].index.to_list()
    if len(index) == 0:
        inDF2notCong[precinctNum] = match
inDF2notCong

{'107': ['Barrington Presidential', 'Bristol County'],
 '211': ['Bristol Presidential', 'Bristol County'],
 '3405': ['Warren Presidential', 'Bristol County'],
 '620': ['Coventry Presidential', 'Kent County'],
 '907': ['East Greenwich Presidential', 'Kent County'],
 '3535': ['Warwick Presidential', 'Kent County'],
 '3706': ['West Greenwich Presidential', 'Kent County'],
 '3812': ['West Warwick Presidential', 'Kent County'],
 '1504': ['Jamestown Presidential', 'Newport County'],
 '1803': ['Little Compton Presidential', 'Newport County'],
 '1907': ['Middletown Presidential', 'Newport County'],
 '2109': ['Newport Limited', 'Newport County'],
 '2110': ['Newport Presidential', 'Newport County'],
 '2710': ['Portsmouth Presidential', 'Newport County'],
 '3309': ['Tiverton Presidential', 'Newport County'],
 '308': ['Burrillville Presidential', 'Providence County'],
 '410': ['Central Falls Presidential', 'Providence County'],
 '733': ['Cranston Presidential', 'Providence County'],
 '815': ['Cumb

In [80]:
inCongNotDF2 = {}
for precinctNum, precinctName in c2pCong.items():
    index = df2[df2["precinct"] == precinctName+"; "+str(precinctNum)].index.to_list()
    if len(index) == 0:
        inCongNotDF2[precinctNum] = precinctName
inCongNotDF2

{2109: 'Newport 2109', 2110: 'Newport Limited'}

In [81]:
# start filling in the county
cong["County"] = ""
for index, row in cong.iterrows():
    precinct1, precinct2 = row["Precinct Name"], str(row["Precinct #"])
    if precinct2 in c2p.keys() and precinct1 in c2p[precinct2]:
        cong.at[index, "County"] = c2p[precinct2][1]

cong["County"].unique()

array(['Bristol County', 'Providence County', 'Newport County', '', nan,
       'Washington County', 'Kent County'], dtype=object)

In [82]:
# fill 2109: newport 2019 with county Newport County
# same for 2110 Newport Limited
index = cong[(cong["Precinct #"] == "2109") & (cong["Precinct Name"] == "Newport 2109")].index.to_list()[0]
cong.at[index, "County"] = "Newport County"
index = cong[(cong["Precinct #"] == "2110") & (cong["Precinct Name"] == "Newport Limited")].index.to_list()[0]
cong.at[index, "County"] = "Newport County"

In [83]:
# check county that is not filled
indexes = cong[cong["County"] == ''].index.to_list()
for index in indexes:
    cong.at[index, "County"] = "Newport County"
cong["County"].unique()

array(['Bristol County', 'Providence County', 'Newport County', nan,
       'Washington County', 'Kent County'], dtype=object)

In [84]:
def normalizePrecinctNum(df):
    df = df.astype({"Precinct #": "str"})
    for index, row in df.iterrows():
        if len(row["Precinct #"]) != 4:
            df.at[index, "Precinct #"] = "0"+row["Precinct #"]
    return df

cong = normalizePrecinctNum(cong)
cong.head(2)

,Precinct Name,Precinct #,Total Votes,E Day,Mail,Party,Office Title,Candidate,Vote For,County
2315,Barrington 0101,0101,916,858,58,DEM,Representative in Congress District 1,David N. Cicilline,1,Bristol County
2316,Barrington 0102,0102,1245,1131,114,DEM,Representative in Congress District 1,David N. Cicilline,1,Bristol County


In [85]:
cong.drop(columns=["Vote For", "E Day", "Mail"], axis=1, inplace=True)
cong.head(2)

,Precinct Name,Precinct #,Total Votes,Party,Office Title,Candidate,County
2315,Barrington 0101,0101,916,DEM,Representative in Congress District 1,David N. Cicilline,Bristol County
2316,Barrington 0102,0102,1245,DEM,Representative in Congress District 1,David N. Cicilline,Bristol County


In [86]:
cong.rename({"Total Votes":"Total"}, inplace=True, axis=1)

In [87]:
cong["Party"].unique()

array(['DEM', 'REP', 'NON'], dtype=object)

# merge by precinct Name

In [88]:
grouped = cong.groupby(["Office Title", "Precinct #", "Precinct Name", "County", "Candidate", "Party"])
grouped.sum()

Total
Office Title                          Precinct # Precinct Name        County         Candidate            Party       
Representative in Congress District 1 0101       Barrington 0101      Bristol County David N. Cicilline   DEM      916
                                                                                     Patrick J. Donovan   REP      494
                                                                                     WRITE-IN             NON        2
                                      0102       Barrington 0102      Bristol County David N. Cicilline   DEM     1245
                                                                                     Patrick J. Donovan   REP      558
...                                                                                                                ...
Representative in Congress District 2 3810       West Warwick 3810    Kent County    Salvatore G. Caiozzo REP      269
                                                                                     WRITE-IN             NON        0
                                      3811       West Warwick Limited Kent County    James R. Langevin    DEM        0
                                                                                     Salvatore G. Caiozzo REP        0
                                                                                     WRITE-IN             NON        0

[1383 rows x 1 columns]

In [89]:
# new dataframe
columns = ["County", "Precinct Name", "Precinct #", "Democratic",
           "Republican", "Independent", "Other", "Total"]
new_RI = pd.DataFrame(columns=columns)
new_RI.head()

,County,Precinct Name,Precinct #,Democratic,Republican,Independent,Other,Total


In [90]:
vote_column_name = {'DEM': "Democratic", 'REP': "Republican", 'NON': "Other", 'Ind': "Independent"}

# iterate the groupby df
print("################## Merging RI Dataframe ##################")

new_row = ""
cur_precinct = ""
total_votes = 0

for index, group in grouped:
    cur_row = group.loc[:].head(1)
    precinct = cur_row["Precinct #"].values[0]

    if precinct != cur_precinct:
        if cur_precinct != "":
            # add TotalVotes to new row
            new_row["Total"] = total_votes
            # add new_row to new_RI dataframe
            new_RI = pd.concat([new_RI, new_row], ignore_index=True, sort=False)
            
        # reset vars
        cur_precinct = precinct
        new_row = ""
        total_votes = 0
    
    if type(new_row) == str:
        # get columns other than Total (Vote)
        new_row = cur_row[["Office Title", "County", "Precinct Name", "Precinct #"]]

    candidate_key = cur_row["Party"].values[0]
    new_row[vote_column_name[candidate_key]] = cur_row["Total"].values[0]
    total_votes += cur_row["Total"].values[0]
    
print("\n################ Done Merging RI Dataframe ###############")

################## Merging RI Dataframe ##################

################ Done Merging RI Dataframe ###############


# match new_RI to real

In [91]:
len(new_RI)

460

In [92]:
new_RI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   County         460 non-null    object
 1   Precinct Name  460 non-null    object
 2   Precinct #     460 non-null    object
 3   Democratic     460 non-null    object
 4   Republican     460 non-null    object
 5   Independent    0 non-null      object
 6   Other          460 non-null    object
 7   Total          460 non-null    object
 8   Office Title   460 non-null    object
dtypes: object(9)
memory usage: 32.5+ KB


In [93]:
new_RI.fillna(value=0, inplace=True)
new_RI.head(1)

,County,Precinct Name,Precinct #,Democratic,Republican,Independent,Other,Total,Office Title
0,Bristol County,Barrington 0101,0101,916,494,0,2,1412,Representative in Congress District 1


In [94]:
def gen_dict(df, plus=0):
    c2p = {}
    for index, row in df.iterrows():
        county = row["County"]
        if plus:
            county += " County"
        precinct1 = row["Precinct #"]
        if county not in c2p.keys():
            c2p[county] = [precinct1]
        else:
            c2p[county].append(precinct1)

    return c2p

In [95]:
realC2P = gen_dict(real, 1)
newC2P = gen_dict(new_RI)

In [96]:
inNewNotReal = {}
inRealNotNew = {}
for county, precList in newC2P.items():
    for prec in precList:
        if prec not in realC2P[county]:
            if county in inNewNotReal.keys():
                inNewNotReal[county].append(prec)
            else:
                inNewNotReal[county] = [prec]
for county, precList in realC2P.items():
    for prec in precList:
        if prec not in newC2P[county]:
            if county in inRealNotNew.keys():
                inRealNotNew[county].append(prec)
            else:
                inRealNotNew[county] = [prec]

In [97]:
print(len(new_RI), len(real), len(new_RI["Precinct #"].unique()), len(real["Precinct #"].unique()))

460 419 460 419


In [98]:
inRealNotNew

{'Providence County': ['0729']}

In [99]:
inNewNotReal

{'Bristol County': ['0106', '0210', '3404'],
 'Providence County': ['0409',
  '0814',
  '1018',
  '1019',
  '1712',
  '2417',
  '2506',
  '2631',
  '2632',
  '2881',
  '3107',
  '3913',
  '0307',
  '0732',
  '1203',
  '1305',
  '1614',
  '2882',
  '3004'],
 'Newport County': ['1503', '1802', '1906', '2109', '2110', '2709', '3308'],
 'Washington County': ['0505',
  '1104',
  '1404',
  '2006',
  '2202',
  '2310',
  '2903',
  '3211',
  '3608'],
 'Kent County': ['0619', '0906', '3534', '3705']}

In [100]:
for county, precs in inNewNotReal.items():
    rows = new_RI[new_RI["County"]==county]
    for index, row in rows.iterrows():
        prec = row["Precinct #"]
        if prec in precs:
            print(county, prec, " Total:", row["Total"])

Bristol County 0106  Total: 0
Bristol County 0210  Total: 0
Bristol County 3404  Total: 0
Providence County 0409  Total: 3
Providence County 0814  Total: 3
Providence County 1018  Total: 522
Providence County 1019  Total: 7
Providence County 1712  Total: 4
Providence County 2417  Total: 2
Providence County 2506  Total: 0
Providence County 2631  Total: 390
Providence County 2632  Total: 6
Providence County 2881  Total: 3
Providence County 3107  Total: 0
Providence County 3913  Total: 0
Providence County 0307  Total: 1
Providence County 0732  Total: 7
Providence County 1203  Total: 3
Providence County 1305  Total: 0
Providence County 1614  Total: 3
Providence County 2882  Total: 5
Providence County 3004  Total: 0
Newport County 1503  Total: 1
Newport County 1802  Total: 0
Newport County 1906  Total: 2
Newport County 2109  Total: 255
Newport County 2110  Total: 3
Newport County 2709  Total: 1
Newport County 3308  Total: 1
Washington County 0505  Total: 0
Washington County 1104  Total: 0
W

In [101]:
for county, precs in inRealNotNew.items():
    rows = real[real["County"]==county.split(" ")[0]]
    for index, row in rows.iterrows():
        prec = row["Precinct #"]
        if prec in precs:
            print(county, prec, " Total Votes:", row["Total Votes"], "Total Population:", row["Total Population"])

Providence County 0729  Total Votes: 0 Total Population: 1577


# fixing
- Providence County 2631 = Providence County 0729
- delete all rest that is in new_RI but not in real

In [102]:
index = new_RI[(new_RI["County"] == "Providence County") & (new_RI["Precinct #"] == "2631")].index
print(index)

Int64Index([144], dtype='int64')


In [103]:
new_RI.at[index[0], "Precinct #"] = "0729"

In [104]:
inNewNotReal = {}
inRealNotNew = {}
realC2P = gen_dict(real, 1)
newC2P = gen_dict(new_RI)
for county, precList in newC2P.items():
    for prec in precList:
        if prec not in realC2P[county]:
            if county in inNewNotReal.keys():
                inNewNotReal[county].append(prec)
            else:
                inNewNotReal[county] = [prec]
for county, precList in realC2P.items():
    for prec in precList:
        if prec not in newC2P[county]:
            if county in inRealNotNew.keys():
                inRealNotNew[county].append(prec)
            else:
                inRealNotNew[county] = [prec]

In [105]:
inRealNotNew

{}

In [106]:
inNewNotReal

{'Bristol County': ['0106', '0210', '3404'],
 'Providence County': ['0409',
  '0814',
  '1018',
  '1019',
  '1712',
  '2417',
  '2506',
  '2632',
  '2881',
  '3107',
  '3913',
  '0307',
  '0732',
  '1203',
  '1305',
  '1614',
  '2882',
  '3004'],
 'Newport County': ['1503', '1802', '1906', '2109', '2110', '2709', '3308'],
 'Washington County': ['0505',
  '1104',
  '1404',
  '2006',
  '2202',
  '2310',
  '2903',
  '3211',
  '3608'],
 'Kent County': ['0619', '0906', '3534', '3705']}

In [107]:
for index, row in new_RI.iterrows():
    county = row["County"]
    precinctNum = row["Precinct #"]
    if county in inNewNotReal.keys() and precinctNum in inNewNotReal[county]:
        new_RI.drop(index=index, axis=0, inplace=True)
len(new_RI)

419

In [108]:
real.head(1)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
2446,Bristol,0101,1027,522,60,23,2,1634,ri,ri-bristol-0101,...,3268,3055,23,3,110,73,None,None,None,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."


In [109]:
def gen_dict(df, plus=0):
    c2p = {}
    for index, row in df.iterrows():
        county = row["County"]
        precinct = row["Precinct #"]
        if county not in c2p.keys():
            c2p[county] = {precinct: row["CName"]}
        else:
            c2p[county][precinct] = row["CName"]

    return c2p
real_CName_dict = gen_dict(real)

In [110]:
real_CName_dict

{'Bristol': {'0101': 'ri-bristol-0101',
  '0102': 'ri-bristol-0102',
  '0103': 'ri-bristol-0103',
  '0104': 'ri-bristol-0104',
  '0105': 'ri-bristol-0105',
  '0201': 'ri-bristol-0201',
  '0202': 'ri-bristol-0202',
  '0203': 'ri-bristol-0203',
  '0204': 'ri-bristol-0204',
  '0205': 'ri-bristol-0205',
  '0206': 'ri-bristol-0206',
  '0207': 'ri-bristol-0207',
  '0208': 'ri-bristol-0208',
  '0209': 'ri-bristol-0209',
  '3401': 'ri-bristol-3401',
  '3402': 'ri-bristol-3402',
  '3403': 'ri-bristol-3403'},
 'Kent': {'0601': 'ri-kent-0601',
  '0602': 'ri-kent-0602',
  '0603': 'ri-kent-0603',
  '0604': 'ri-kent-0604',
  '0605': 'ri-kent-0605',
  '0606': 'ri-kent-0606',
  '0607': 'ri-kent-0607',
  '0608': 'ri-kent-0608',
  '0609': 'ri-kent-0609',
  '0610': 'ri-kent-0610',
  '0611': 'ri-kent-0611',
  '0612': 'ri-kent-0612',
  '0613': 'ri-kent-0613',
  '0614': 'ri-kent-0614',
  '0615': 'ri-kent-0615',
  '0616': 'ri-kent-0616',
  '0617': 'ri-kent-0617',
  '0618': 'ri-kent-0618',
  '0901': 'ri-kent-

In [111]:
new_RI["CName"] = ""
for index, row in new_RI.iterrows():
    new_RI.at[index, "CName"] = real_CName_dict[row["County"].split(" ")[0]][row["Precinct #"]]
new_RI.head()

,County,Precinct Name,Precinct #,Democratic,Republican,Independent,Other,Total,Office Title,CName
0,Bristol County,Barrington 0101,0101,916,494,0,2,1412,Representative in Congress District 1,ri-bristol-0101
1,Bristol County,Barrington 0102,0102,1245,558,0,8,1811,Representative in Congress District 1,ri-bristol-0102
2,Bristol County,Barrington 0103,0103,991,471,0,4,1466,Representative in Congress District 1,ri-bristol-0103
3,Bristol County,Barrington 0104,0104,1176,567,0,0,1743,Representative in Congress District 1,ri-bristol-0104
4,Bristol County,Barrington 0105,0105,1186,583,0,7,1776,Representative in Congress District 1,ri-bristol-0105


In [112]:
new_RI.drop(columns=["County", "Precinct Name", "Precinct #"], axis=1, inplace=True)
new_RI.rename({"Office Title":"dist_num"}, axis=1, inplace=True)
new_RI.head(2)

,Democratic,Republican,Independent,Other,Total,dist_num,CName
0,916,494,0,2,1412,Representative in Congress District 1,ri-bristol-0101
1,1245,558,0,8,1811,Representative in Congress District 1,ri-bristol-0102


In [113]:
for index, row in new_RI.iterrows():
    new_RI.at[index, "dist_num"] = row["dist_num"].split(" ")[-1]
new_RI["dist_num"].unique()

array(['1', '2'], dtype=object)

In [114]:
new_RI["Type"] = "CONG18"

In [115]:
new_RI.head()

,Democratic,Republican,Independent,Other,Total,dist_num,CName,Type
0,916,494,0,2,1412,1,ri-bristol-0101,CONG18
1,1245,558,0,8,1811,1,ri-bristol-0102,CONG18
2,991,471,0,4,1466,1,ri-bristol-0103,CONG18
3,1176,567,0,0,1743,1,ri-bristol-0104,CONG18
4,1186,583,0,7,1776,1,ri-bristol-0105,CONG18


In [116]:
new_RI.isna().sum()

Democratic     0
Republican     0
Independent    0
Other          0
Total          0
dist_num       0
CName          0
Type           0
dtype: int64

In [117]:
len(new_RI)

419

In [118]:
len(real)

419

In [119]:
new_RI.to_csv(file_name.split(".")[0]+".csv", index=False)